<a href="https://colab.research.google.com/github/manuelfernandezmontejano/riskregulatorypublications/blob/main/WebScrapping_Regulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
# Importa las bibliotecas necesarias
!apt-get update
!apt-get install -y wkhtmltopdf
!which wkhtmltopdf
!pip install pdfkit
!pip install filestack-python
import requests
from bs4 import BeautifulSoup
import os
import re
import pdfkit
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import filestack
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import time

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,705 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,662 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 11.6 MB in 2s (6,118 kB/s)
Reading package lists... Done
W: Skipping acquire of configured fi

In [78]:
# Define un diccionario con los publicadores y sus enlaces
enlaces_publicadores = {
  "ECB": "https://www.bankingsupervision.europa.eu/home/html/index.en.html",
  "BCBS publications": "https://www.bis.org/bcbs/index.htm",
  "FSB publications": "https://www.fsb.org/",
  "EBA press release": "https://www.eba.europa.eu/publications-and-media/press-releases",
  "CNMV sala de prensa": "https://www.cnmv.es/portal/AlDia/Comunicaciones-Publicas",
  "ESMA press release": "https://www.esma.europa.eu/press-news/press-releases",
  "BdE publicaciones": "https://www.bde.es/wbe/es/publicaciones/",
  "BdE noticias": "https://www.bde.es/wbe/es/noticias-eventos/",
  "DG FISMA latest": "https://commission.europa.eu/about/departments-and-executive-agencies/financial-stability-financial-services-and-capital-markets-union_en",
  "DOUE legislacion": "https://eur-lex.europa.eu/oj/direct-access.html?locale=es",
  "PRA news and publications": "https://www.bankofengland.co.uk/news",
  "EFRAG updates": "https://www.efrag.org/en/updates",
  "FRB news": "https://www.federalreserve.gov/newsevents.htm",
  "ESG today": "https://www.esgtoday.com/category/business-wire/"
}

In [79]:
# Crea una carpeta para guardar los resultados
if not os.path.exists("resultados"):
    os.makedirs("resultados")

In [80]:
# Recorre el diccionario y hace web scraping de cada página
for publicador, enlace in enlaces_publicadores.items():
    print(f"Procesando: {publicador} - {enlace}")

    try:
        # Obtener contenido de la página
        respuesta = requests.get(enlace, timeout=10)

        # Verificar si la respuesta es exitosa (código 200)
        if respuesta.status_code != 200:
            print(f"Error {respuesta.status_code} al obtener {publicador}")
            continue

        # Analizar el HTML
        soup = BeautifulSoup(respuesta.text, "html.parser")

        # Extraer solo el texto sin HTML
        contenido = soup.get_text(separator="\n", strip=True)

        # Formatear nombre de archivo
        nombre_archivo = re.sub(r"[^a-zA-Z0-9]", "_", publicador) + ".txt"
        ruta_archivo = os.path.join("resultados", nombre_archivo)

        # Guardar en archivo con encabezado
        with open(ruta_archivo, "w", encoding="utf-8") as archivo:
            archivo.write(f"{publicador} ({enlace})\n\n")
            archivo.write(contenido)

        print(f"Guardado: {ruta_archivo}")

    except requests.RequestException as e:
        print(f"Error al procesar {publicador}: {e}")

print("Proceso finalizado.")

Procesando: ECB - https://www.bankingsupervision.europa.eu/home/html/index.en.html
Guardado: resultados/ECB.txt
Procesando: BCBS publications - https://www.bis.org/bcbs/index.htm
Guardado: resultados/BCBS_publications.txt
Procesando: FSB publications - https://www.fsb.org/
Guardado: resultados/FSB_publications.txt
Procesando: EBA press release - https://www.eba.europa.eu/publications-and-media/press-releases
Guardado: resultados/EBA_press_release.txt
Procesando: CNMV sala de prensa - https://www.cnmv.es/portal/AlDia/Comunicaciones-Publicas
Guardado: resultados/CNMV_sala_de_prensa.txt
Procesando: ESMA press release - https://www.esma.europa.eu/press-news/press-releases
Guardado: resultados/ESMA_press_release.txt
Procesando: BdE publicaciones - https://www.bde.es/wbe/es/publicaciones/
Guardado: resultados/BdE_publicaciones.txt
Procesando: BdE noticias - https://www.bde.es/wbe/es/noticias-eventos/
Guardado: resultados/BdE_noticias.txt
Procesando: DG FISMA latest - https://commission.europ

In [81]:
# Combina los archivos de texto en un solo archivo HTML
html_combinado = ""
for nombre_archivo in os.listdir("resultados"):
    if nombre_archivo.endswith(".txt"):
        publicador = nombre_archivo[:-4].replace("_", " ")
        enlace = enlaces_publicadores.get(publicador, "")  # Obtiene el enlace del diccionario
        ruta_archivo = os.path.join("resultados", nombre_archivo)
        with open(ruta_archivo, "r", encoding="utf-8") as archivo:
            # Incluye el publicador y el enlace en el título
            html_combinado += f"<h1>{publicador} ({enlace})</h1>\n"
            html_combinado += f"<p>{archivo.read()}</p>\n"

# Guarda el HTML combinado en un archivo
with open("resultados/combinado.html", "w", encoding="utf-8") as archivo:
    archivo.write(html_combinado)

In [82]:
config = pdfkit.configuration(wkhtmltopdf='/usr/bin/wkhtmltopdf')  # Changed path to where wkhtmltopdf is actually located
pdfkit.from_file("resultados/combinado.html", "resultados/combinado.pdf", configuration=config)

True

In [83]:
def send_pdf_via_filestack(pdf_path):
    """
    Uploads the PDF to Filestack and returns the public link.

    Args:
        pdf_path: Path to the PDF file.

    Returns:
        public_url: Public URL of the uploaded file.
    """
    import filestack

    # Reemplaza con tu clave de API de Filestack
    client = filestack.Client('A3Yi51FVPSASBv8QXS16Ez')

    # Subir el archivo
    filelink = client.upload(filepath=pdf_path)

    # Obtener el enlace público
    public_url = filelink.url

    print(f"Public link to PDF: {public_url}")

    return public_url  # Retorna el enlace


def send_pdf_via_email(pdf_path, email_address, sender_email, sender_password, public_url=None):
    """
    Sends the PDF as an email attachment or includes a Filestack link.

    Args:
        pdf_path: Path to the PDF file.
        email_address: Recipient's email address.
        sender_email: Sender's email address.
        sender_password: Sender's email password.
        public_url: Optional public link to the PDF file.
    """
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders

    msg = MIMEMultipart()
    msg['Subject'] = "Regulatory Updates PDF"
    msg['From'] = sender_email
    msg['To'] = email_address

    # Mensaje de correo con enlace de Filestack (si existe)
    html = f"""\
    <html>
      <head></head>
      <body>
        <p>Please find the attached Regulatory Updates PDF.</p>
        {"<p>Download it from: <a href='" + public_url + "'>this link</a></p>" if public_url else ""}
      </body>
    </html>
    """

    msg.attach(MIMEText(html, 'html'))

    # Adjuntar el PDF solo si no hay un link de Filestack
    if not public_url:
        with open(pdf_path, "rb") as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f"attachment; filename= {pdf_path}")
            msg.attach(part)

    # Enviar correo
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, email_address, msg.as_string())

    print(f"Email sent to {email_address}")


# **Ejemplo de uso corregido**
pdf_path = "resultados/combinado.pdf"  # Ruta del PDF generado
recipient_email = "manuel.fernandez.montejano@bbva.com"
sender_email = "manuel.fernandez.montejano@bbva.com"
sender_password = "hehrjrdxzdsgunnl"

# Subir a Filestack y obtener enlace
public_url = send_pdf_via_filestack(pdf_path)

# Enviar email con adjunto y/o enlace de Filestack
send_pdf_via_email(pdf_path, recipient_email, sender_email, sender_password, public_url)


Public link to PDF: https://cdn.filestackcontent.com/rzjg19kVQaCTdW4g6Ihp
Email sent to manuel.fernandez.montejano@bbva.com
